In [231]:
import boto3
from trp import Document
import cv2
country_dict = {} 

from googletrans import Translator, constants
translator = Translator()

from fuzzywuzzy import fuzz

In [232]:
card_identification = {'croatia' : {'country':'Croatia',
                                            'Types_of_card':['identity card','driving licence']},
                    'france' : {'country':'France',
                                            'Types_of_card':['identity card','driving licence']},
                    'belgium' : {'country':'Belgium',
                                            'Types_of_card':['identity card','driving licence']},
                    'germany' : {'country':'Germany',
                                            'Types_of_card':['identity card','driving licence']},
                    'espana' : {'country':'Spain',
                                            'Types_of_card':['identity card','driving licence']},
                    'poland' : {'country':'Poland',
                                            'Types_of_card':['identity card','driving licence']},
                    'romania' : {'country':'Romania',
                                            'Types_of_card':['identity card','driving licence']},
                    'slovenia' : {'country':'Slovenia',
                                            'Types_of_card':['identity card','driving licence']},
                    'turkey' : {'country':'Turkey',
                                            'Types_of_card':['identity card','driving licence']},
                    'india' : {'country':'India',
                                            'Types_of_card':['adhar','driving licence', 'pan card']}}

In [233]:
# S3 Bucket Data
def extract_top(file_path,s3BucketName):
    # s3BucketName = "kshatra-ai-jaydev"
    # file_path = "./id_card_imgs/croatia3.jpeg"
    img = cv2.imread(file_path)
    img = cv2.resize(img, (1260, 820))
    list = file_path.split('/')
    cv2.imwrite(file_path, img)
    file_name = list[-1]


    s3 = boto3.client('s3')
    s3.upload_file(file_path ,s3BucketName, file_name)
    print(".....DONE -UPLOADING.......")
    FormdocumentName = file_name
    PlaindocumentName = file_name
    # Amazon Textract client
    textractmodule = boto3.client('textract')
    top_list = []
    top_list_eng = []
    #1. PLAINTEXT detection from documents:
    response = textractmodule.detect_document_text(Document={'S3Object': {'Bucket': s3BucketName,'Name': PlaindocumentName}})

    print ('------------- Print Plaintext detected text ------------------------------')
    i = 0
    for item in response["Blocks"]:
        if item["BlockType"] == "WORD":
            top_list.append(str(item['Text']).lower())
            key = translator.translate(str(item['Text']))
            ket_txt = key.text.lower()
            top_list_eng.append(ket_txt)
    # print(top_list)
    # print(top_list_eng)
    return top_list,top_list_eng

In [242]:
top_list, top_list_eng = extract_top( file_path = "./../id_card_imgs/ukrain3.jpeg", s3BucketName = "kshatra-ai-jaydev")

.....DONE -UPLOADING.......
------------- Print Plaintext detected text ------------------------------


In [235]:
def identify_country(card_identification, top_list_eng):
    max_fuzzy = 0
    cntry = "Not Found."
    a = 0
    for kkeeyy in card_identification:
        for a in range (15):
            fuzz_value = fuzz.ratio(top_list_eng[a], kkeeyy)
            # print(top_list_eng[a],kkeeyy)
            # print(fuzz_value)
            if fuzz_value > max_fuzzy:
                if fuzz_value > 50:
                    max_fuzzy = fuzz_value
                    cntry = kkeeyy
            types_of_card =  card_identification[kkeeyy]['Types_of_card']
            
    print(max_fuzzy)
    return cntry, types_of_card ,top_list_eng

In [236]:
# country, types_of_card = identify_country(card_identification, top_list_eng)

In [239]:
def identify_card(card_identification, top_list_eng):
    country, card_types, top_list_eng= identify_country(card_identification, top_list_eng)
    # print(card_types)
    max_fuzzy =0
    card_type = "Not Found."
    for c in range (len(top_list_eng)):
        for b in range(len(card_types)):
            fuzz_value = fuzz.ratio(top_list_eng[c], card_types[b])
            if fuzz_value > max_fuzzy:
                if fuzz_value > 50:
                    max_fuzzy = fuzz_value
                    card_type = card_types[b]
            
    return country,card_type
    # print("Card of card does not exist in dict.")


In [240]:
country, card = identify_card(card_identification, top_list_eng)
if country == "Not Found.":
    print("country Not Found, add this card in Dataset.")

62


In [241]:
country, card

('france', 'Not Found.')

In [227]:
key_value = 'pakistan'
country_name = 'Pakistan'
new_card_type = 'identity card'
card_type_list = []

In [228]:
card_type_list.append(new_card_type)
new_data = {key_value:{'country': country_name,'Types_of_card':card_type_list}}

In [229]:
card_identification.update(new_data)


In [230]:
card_identification

{'croatia': {'country': 'Croatia',
  'Types_of_card': ['identity card', 'driving licence']},
 'france': {'country': 'France',
  'Types_of_card': ['identity card', 'driving licence']},
 'belgium': {'country': 'Belgium',
  'Types_of_card': ['identity card', 'driving licence']},
 'germany': {'country': 'Germany',
  'Types_of_card': ['identity card', 'driving licence']},
 'espana': {'country': 'Spain',
  'Types_of_card': ['identity card', 'driving licence']},
 'poland': {'country': 'Poland',
  'Types_of_card': ['identity card', 'driving licence']},
 'romania': {'country': 'Romania',
  'Types_of_card': ['identity card', 'driving licence']},
 'slovenia': {'country': 'Slovenia',
  'Types_of_card': ['identity card', 'driving licence']},
 'turkey': {'country': 'Turkey',
  'Types_of_card': ['identity card', 'driving licence']},
 'india': {'country': 'India',
  'Types_of_card': ['adhar card', 'driving licence', 'pan card']},
 'pakistan': {'country': 'Pakistan', 'Types_of_card': ['identity card']}